# Word embeddings

In [1]:
import datasets # Biblioteca de manejo de conjuntos de datos para procesamiento de lenguaje natural
import mae2 # Biblioteca del curso donde iremos guardando funciones importantes
import gensim # Biblioteca de modelamiento de lenguaje
import numpy as np # Biblioteca de manejo de datos vectoriales
import sklearn.linear_model # Módulo de sklearn de modelamiento lineal
import sklearn.decomposition # Módulo de sklearn donde está PCA
import plotly.express as px # Biblioteca de visualización
import urllib # Módulo de python para el manejo de consultas web

Cargamos el conjunto de datos del curso.

In [2]:
spanish_diagnostics = datasets.load_dataset('fvillena/spanish_diagnostics') # Cargamos las particiones de entrenamiento y prueba

Preprocesamos el corpus.

In [3]:
spanish_diagnostics_normalized = spanish_diagnostics.map(
    lambda x: { 
        "normalized_text" : mae2.normalize(x["text"]) 
    })

In [4]:
spanish_diagnostics_normalized_tokenized = spanish_diagnostics_normalized.map(
    lambda x: { 
        "tokenized_text" : x["normalized_text"].split()
    })

## Word embeddings

Los word embeddings son representaciones vectoriales de palabras. A cada una de las palabras de nuestro vocabulario se les asigna un vector que la representa y estas representaciones guardan relaciones semánticas del lenguaje natural. Por ejemplo, palabras semánticamente similares tienen a estar más cerca en el espacio que palabras no tan similares entre sí.

Para calcular estas representaciones utilizaremos el método Word2Vec, en su implementación de la biblioteca Gensim.

Para calcular las representaciones, simplemente pasamos nuestro corpus tokenizado al objeto Word2Vec.

In [5]:
model = gensim.models.Word2Vec(spanish_diagnostics_normalized_tokenized["train"]["tokenized_text"])

Tenemos una representación de 100 dimensiones para cada una de las palabras del vocabulario.

In [6]:
model.wv.vectors.shape

(6316, 100)

Si extraemos el vector asociado a una palabra se observa que está constituido de números reales.

In [7]:
model.wv["pieza"]

array([ 2.4565172e-01, -1.0703677e+00,  1.5897071e-01, -7.7539492e-01,
       -1.2157948e+00, -4.0518191e-01, -1.0728382e+00,  9.8960882e-01,
       -8.8930064e-01, -2.4495552e+00,  6.7282476e-02,  9.3205744e-01,
       -1.2039865e+00, -2.8598198e-01,  1.2068130e+00,  7.2707474e-01,
       -1.4746871e+00, -1.9961350e+00, -5.0894034e-01, -8.5245717e-01,
       -1.2063965e+00, -4.0731087e-01,  3.4414884e-01,  1.4990951e+00,
       -6.1525369e-01,  5.1699257e-01, -2.1107303e-01,  9.1114372e-01,
        6.6333419e-01,  1.5787313e+00,  2.0173354e+00, -1.5839072e+00,
        1.3688259e+00, -5.0054383e-01,  5.4976732e-01,  4.3965396e-01,
        7.8627366e-01,  1.9371257e+00,  7.7057499e-01, -6.1945552e-01,
        9.7263902e-01, -1.6128968e+00,  9.4676271e-02,  3.7781546e-01,
       -6.9840908e-01, -4.3995783e-01,  1.6301770e+00, -4.1414413e-01,
        1.3577473e+00,  9.5802230e-01, -2.5905177e-01,  1.4778247e-02,
       -8.4008712e-01,  5.5288994e-01, -4.3806928e-01, -3.0854899e-01,
      

Una de las métricas más utilizadas para establecer distancias entre vectores de palabras es la similaridad coseno, esta métrica está por defecto utilizada cuando deseamos saber las palabras más cercanas a un palabra que consultamos. Se observa que las palabras más cercanas a la palabra consultada guardan una estrecha relación semántica.

### Similaridad

In [8]:
model.wv.most_similar("pieza")

[('pza', 0.9383518695831299),
 ('pd', 0.8728413581848145),
 ('pz', 0.8548220992088318),
 ('dte', 0.8192819952964783),
 ('diente', 0.8055036664009094),
 ('piezas', 0.7771077752113342),
 ('pzas', 0.7685713171958923),
 ('raiz', 0.7061225771903992),
 ('restauracion', 0.7050464749336243),
 ('corona', 0.6940282583236694)]

Para continuar con la verificación de la similaridad de palabras establecemos pares de palabras en donde cada vez iremos alejándolas semánticamente. También podemos concluir que nuestro modelo pudo satisfactoriamente representar estas distancias de manera correcta.

In [9]:
pairs = [
    ('pieza', 'diente'), # Palabras muy cercanas semánticamente
    ('pieza', 'caries'),
    ('pieza', 'protesis'),
    ('pieza', 'hueso'),
    ('pieza', 'sangre') # Palabras muy lejanas semanticamente
]

In [10]:
for w1, w2 in pairs:
    print('%r\t%r\t%.2f' % (w1, w2, model.wv.similarity(w1, w2)))

'pieza'	'diente'	0.81
'pieza'	'caries'	0.58
'pieza'	'protesis'	0.31
'pieza'	'hueso'	0.10
'pieza'	'sangre'	0.14


Una de las aplicaciones de la similaridad de palabras es la detección de palabras fuera de contexto en una lista de palabras. En este caso el modelo detectó correctamente la palabra extraña.

In [11]:
model.wv.doesnt_match(["diente","periodontitis","cirrosis","corona","lengua"])

'cirrosis'

### Analogía

> Analogía, significa comparación o relación entre varias cosas, razones o conceptos; comparar o relacionar dos o más seres u objetos a través de la razón; señalando características generales y particulares comunes que permiten justificar la existencia de una propiedad en uno, a partir de la existencia de dicha propiedad en los otros.
>
>En el aspecto lógico, permite comparar un objeto con otros, en sus semejanzas y en sus diferencias. Una analogía permite la deducción de un término desconocido a partir del análisis de la relación que se establece entre dos términos conocidos.
>
> Analogía - Wikipedia

Tomando en cuenta esta definición podemos construir analogías como las siguientes:

* *pieza* es a *caries* como *ojo* es a *catarata*
* *bacteria* es a *antibiótico* como *virus* es a *antiviral*

Si a estos cuartetos de palabras le quitamos una, podemos transformar estas analogías en preguntas de analogía y esta es una tarea en la cual los word embeddings típicamente son probados:

* *pieza* es a *caries* como *ojo* es a *¿?*
* *bacteria* es a *antibiótico* como *virus* es a *¿?*

La relación vectorial que podemos construir para resolver estas pruebas es la siguiente: 

Siendo $w_1$, $w_2$, $w_3$, $w_4$ los vectores asociados a las 4 palabras de la analogía, se cumple que $w_2 - w_1 	\approx w_4 - w_3$, por lo que para resolver una prueba de analogía, la posición de la palabra incógnita $w_4$ es $w_2 - w_1 + w_3$.

Aquí vemos como nuestro modelo resuelve correctamente una prueba de analogía.

In [12]:
w_1 = model.wv["pieza"]
w_2 = model.wv["caries"]
w_3 = model.wv["ojo"]
w_4 = w_2 - w_1 + w_3

model.wv.similar_by_vector(w_4)

[('ojo', 0.6487950086593628),
 ('catarata', 0.6474996209144592),
 ('pterigion', 0.6205140948295593),
 ('glaucoma', 0.6148248910903931),
 ('cataratas', 0.5835525393486023),
 ('vicio', 0.5735694766044617),
 ('oi', 0.5700180530548096),
 ('vicios', 0.5492902994155884),
 ('estrabismo', 0.5409233570098877),
 ('borrosa', 0.5373815894126892)]

También gensim implementa el método Word2Vec.most_similar() que nos ayuda a resolver en 1 línea esta operación.

In [13]:
model.wv.most_similar(positive=["caries", "ojo"], negative=["pieza"])

[('catarata', 0.7039384841918945),
 ('pterigion', 0.6901866793632507),
 ('glaucoma', 0.682616114616394),
 ('cataratas', 0.6427133083343506),
 ('oi', 0.623244047164917),
 ('vicio', 0.61155766248703),
 ('borrosa', 0.6090128421783447),
 ('ocular', 0.5996009111404419),
 ('vicios', 0.5947058200836182),
 ('estrabismo', 0.5854259133338928)]

### Visualización

Para poder visualizar nuestro espacio vectorial debemos proyectar los embeddings en 2 dimensiones, para lo cual utilizaremos un método de reducción de dimensionalidad

In [14]:
projector = sklearn.decomposition.PCA(3)
vectors_2d = projector.fit_transform(model.wv.vectors)[:,:2]

Podemos explorar en un espacio bidimensional las relaciones de cada una de nuestras palabras.

In [15]:
fig = px.scatter(
    x=vectors_2d[:,0],
    y=vectors_2d[:,1],
    text=model.wv.index_to_key
)
fig.update_traces(mode="markers")

### Comparación

Existen word embeddings precalculados disponibles libremente en internet. Utilizaremos como embedding de dominio general el Spanish Billion Words Corpus Embeddings y compararemos el rendimiento sobre pruebas clínicas de los 2 embeddings.

Cargamos los embeddings.

In [16]:
sbwce = gensim.models.KeyedVectors.load_word2vec_format("/workspaces/mae2/data/sbwce_slim.txt")

Podemos observar que si probamos este modelo sobre palabras del dominio clínico, los resultados de estas pruebas son desfavorables.

In [17]:
sbwce.most_similar("pieza")

[('piezas', 0.6851319074630737),
 ('obra', 0.5722230672836304),
 ('cuerda', 0.5271234512329102),
 ('figura', 0.49860382080078125),
 ('máquina', 0.48856663703918457),
 ('elemento', 0.47292402386665344),
 ('clave', 0.47156471014022827),
 ('escultura', 0.46959367394447327),
 ('montaje', 0.46599438786506653),
 ('estructura', 0.4569995105266571)]

In [18]:
for w1, w2 in pairs:
    try:
        print('%r\t%r\t%.2f' % (w1, w2, sbwce.similarity(w1, w2)))
    except KeyError as e:
        print(e)

"Key 'diente' not present"
"Key 'caries' not present"
"Key 'protesis' not present"
"Key 'hueso' not present"
'pieza'	'sangre'	0.21


In [19]:
sbwce.doesnt_match(["diente","periodontitis","cirrosis","corona","lengua"])

'corona'

Probamos una analogía típica utilizada para verificar el rendimiento de los modelos de dominio general.

In [20]:
sbwce.most_similar(positive=["mujer", "rey"], negative=["hombre"], topn=3)

[('reina', 0.7493032217025757),
 ('princesa', 0.6861547231674194),
 ('reyes', 0.6391469836235046)]

In [21]:
sbwce_2d = projector.fit_transform(sbwce.vectors)[:,:2]

In [22]:
fig = px.scatter(
    x=sbwce_2d[:,0],
    y=sbwce_2d[:,1],
    text=list(sbwce.index_to_key)
)
fig.update_traces(mode="markers")

## Clasificación de textos

Definimos una función que nos retornará una representación de un documento desde representaciones de palabras.

In [23]:
def to_vector(tokens,model):
    """ Receives a sentence string along with a word embedding model and 
    returns the vector representation of the sentence"""
    vec = np.zeros(model.wv.vectors.shape[1]) # creates an empty vector of 300 dimensions
    for word in tokens: # iterates over the sentence
        if word in model.wv: # checks if the word is both in the word embedding and the tf-idf model
            vec += model.wv[word] # adds every word embedding to the vector
    if np.linalg.norm(vec) > 0:
        return vec / np.linalg.norm(vec) # divides the vector by their normal
    else:
        return vec

Utilizaremos la función definida anteriormente para obtener la representación vectorial de cada uno de los ejemplos de nuestro conjunto de datos.

In [24]:
spanish_diagnostics_normalized_tokenized_vectorized = spanish_diagnostics_normalized_tokenized.map(
    lambda x: { 
        "vectorized_text" : to_vector(x["tokenized_text"],model)
    })

Map:   0%|          | 0/70000 [00:00<?, ? examples/s]

Map:   0%|          | 0/30000 [00:00<?, ? examples/s]

Exploramos el espacio y verificamos que existe una separación relativamente clara entre los documentos de las distintas clases.

In [25]:
examples_3d = projector.fit_transform(spanish_diagnostics_normalized_tokenized_vectorized["test"]["vectorized_text"])

In [26]:
fig = px.scatter_3d(
    x=examples_3d[:,0],
    y=examples_3d[:,1],
    z=examples_3d[:,2],
    color=spanish_diagnostics_normalized_tokenized_vectorized["test"].features["label"].int2str(spanish_diagnostics_normalized_tokenized_vectorized["test"]["label"]),
    opacity=0.5
)
fig.update_traces(mode="markers",marker={'size':2})

Instanciamos y ajustamos un modelo de regresión logística para clasificar nuestros textos y vemos que este método tiene un rendimiento significativamente bueno

In [27]:
classifier = sklearn.linear_model.LogisticRegression(solver='liblinear')

In [28]:
classifier.fit(
    spanish_diagnostics_normalized_tokenized_vectorized["train"]["vectorized_text"],
    spanish_diagnostics_normalized_tokenized_vectorized["train"]["label"]
)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'liblinear'
,max_iter,100
,multi_class,'deprecated'


In [29]:
print(sklearn.metrics.classification_report(
    spanish_diagnostics_normalized_tokenized_vectorized["test"]["label"],
    classifier.predict(spanish_diagnostics_normalized_tokenized_vectorized["test"]["vectorized_text"])
))

              precision    recall  f1-score   support

           0       0.93      0.94      0.94     15034
           1       0.94      0.93      0.94     14966

    accuracy                           0.94     30000
   macro avg       0.94      0.94      0.94     30000
weighted avg       0.94      0.94      0.94     30000



Realizamos el mismo proceso de vectorización pero utilizando las representaciones de dominio general y verificamos que el rendimiento es mucho menor al anterior.

In [30]:
#esta funcion es igual a to_vector pero con modificaciones para KeyedVectors
def to_vector_2(tokens,model):
    """ Receives a sentence string along with a word embedding model and 
    returns the vector representation of the sentence"""
    vec = np.zeros(model.vectors.shape[1]) # creates an empty vector of 300 dimensions
    for word in tokens: # iterates over the sentence
        if word in model: # checks if the word is both in the word embedding and the tf-idf model
            vec += model[word] # adds every word embedding to the vector
    if np.linalg.norm(vec) > 0:
        return vec / np.linalg.norm(vec) # divides the vector by their normal
    else:
        return vec

In [31]:
spanish_diagnostics_normalized_tokenized_vectorized_sbwce = spanish_diagnostics_normalized_tokenized.map(
    lambda x: { 
        "vectorized_text" : to_vector_2(x["tokenized_text"],sbwce)
    })

Map:   0%|          | 0/70000 [00:00<?, ? examples/s]

Map:   0%|          | 0/30000 [00:00<?, ? examples/s]

In [32]:
examples_3d_sbwce = projector.fit_transform(spanish_diagnostics_normalized_tokenized_vectorized_sbwce["test"]["vectorized_text"])

In [33]:
fig = px.scatter_3d(
    x=examples_3d_sbwce[:,0],
    y=examples_3d_sbwce[:,1],
    z=examples_3d_sbwce[:,2],
    color=spanish_diagnostics_normalized_tokenized_vectorized_sbwce["test"].features["label"].int2str(spanish_diagnostics_normalized_tokenized_vectorized_sbwce["test"]["label"]),
    opacity=0.5
)
fig.update_traces(mode="markers",marker={'size':2})

In [34]:
classifier_sbwce = sklearn.linear_model.LogisticRegression(solver='liblinear')

In [35]:
classifier.fit(
    spanish_diagnostics_normalized_tokenized_vectorized_sbwce["train"]["vectorized_text"],
    spanish_diagnostics_normalized_tokenized_vectorized_sbwce["train"]["label"]
)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'liblinear'
,max_iter,100
,multi_class,'deprecated'


In [36]:
print(sklearn.metrics.classification_report(
    spanish_diagnostics_normalized_tokenized_vectorized_sbwce["test"]["label"],
    classifier.predict(spanish_diagnostics_normalized_tokenized_vectorized_sbwce["test"]["vectorized_text"])
))

              precision    recall  f1-score   support

           0       0.76      0.90      0.82     15034
           1       0.87      0.72      0.79     14966

    accuracy                           0.81     30000
   macro avg       0.82      0.81      0.81     30000
weighted avg       0.82      0.81      0.81     30000

